In [ ]:
# 03_real_world_maps.ipynb

import pandas as pd
import folium
from folium import PolyLine
from typing import List


def load_locations(filepath: str) -> pd.DataFrame:
    """
    Load delivery locations from CSV.

    Args:
        filepath (str): Path to locations CSV.

    Returns:
        pd.DataFrame: DataFrame with 'id', 'latitude', 'longitude', 'address'.
    """
    locations = pd.read_csv(filepath)
    required_cols = {"id", "latitude", "longitude", "address"}
    if not required_cols.issubset(locations.columns):
        raise ValueError(f"CSV file must contain columns: {required_cols}")
    return locations


def load_route(filepath: str) -> List[int]:
    """
    Load optimized route from JSON or text file as a list of location indices or IDs.

    Args:
        filepath (str): Path to route file.

    Returns:
        List[int]: Ordered list of location IDs or indices.
    """
    import json
    with open(filepath, "r") as file:
        route = json.load(file)
    if not isinstance(route, list):
        raise ValueError("Route file must contain a list of location IDs or indices.")
    return route


def plot_route_map(
    locations: pd.DataFrame, 
    route: List[int], 
    map_center: List[float] = None,
    zoom_start: int = 12
) -> folium.Map:
    """
    Plot the delivery route on a Folium map with markers and route lines.

    Args:
        locations (pd.DataFrame): DataFrame with delivery locations.
        route (List[int]): Ordered list of location IDs or indices.
        map_center (List[float], optional): Initial map center coordinates. Defaults to mean.
        zoom_start (int, optional): Initial zoom level.

    Returns:
        folium.Map: Interactive map with route visualization.
    """
    if map_center is None:
        map_center = [locations['latitude'].mean(), locations['longitude'].mean()]

    route_map = folium.Map(location=map_center, zoom_start=zoom_start)

    # Extract route locations in order
    route_locs = locations.set_index('id').loc[route]

    # Add markers with popups
    for idx, row in route_locs.iterrows():
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=f"ID: {row['id']}<br>Address: {row['address']}",
            tooltip=f"Stop {route.index(row['id']) + 1}",
            icon=folium.Icon(color='blue', icon='truck', prefix='fa')
        ).add_to(route_map)

    # Draw the route line
    points = route_locs[['latitude', 'longitude']].values.tolist()
    PolyLine(points, color="red", weight=5, opacity=0.8).add_to(route_map)

    return route_map


# Main execution
if __name__ == "__main__":
    locations_path: str = "data/locations.csv"
    route_path: str = "app/route_result.json"

    locations_df: pd.DataFrame = load_locations(locations_path)
    optimized_route: List[int] = load_route(route_path)

    # Plot map
    delivery_route_map = plot_route_map(locations_df, optimized_route)
    delivery_route_map.save("output/optimized_delivery_route_map.html")

    print("Optimized delivery route map saved to output/optimized_delivery_route_map.html")
